In [246]:
import pandas as pd
import re
from email import policy
from email.parser import BytesParser
from bs4 import BeautifulSoup
from email import message_from_string


In [247]:
data = pd.read_csv("C:/Users/bechi/Downloads/archive/emails.csv", nrows=70000)

In [248]:
data['message'][0]


"Message-ID: <18782981.1075855378110.JavaMail.evans@thyme>\nDate: Mon, 14 May 2001 16:39:00 -0700 (PDT)\nFrom: phillip.allen@enron.com\nTo: tim.belden@enron.com\nSubject: \nMime-Version: 1.0\nContent-Type: text/plain; charset=us-ascii\nContent-Transfer-Encoding: 7bit\nX-From: Phillip K Allen\nX-To: Tim Belden <Tim Belden/Enron@EnronXGate>\nX-cc: \nX-bcc: \nX-Folder: \\Phillip_Allen_Jan2002_1\\Allen, Phillip K.\\'Sent Mail\nX-Origin: Allen-P\nX-FileName: pallen (Non-Privileged).pst\n\nHere is our forecast\n\n "

In [249]:

def clean_email(email_str):
    msg = message_from_string(email_str)
    
    from_address = msg.get('From')
    to_address = msg.get('To')
    subject = msg.get('Subject')
    date = msg.get('Date')
    body = ""   
    if msg.is_multipart():
        for part in msg.get_payload():
            if part.get_content_type() == 'text/html':
                html_content = part.get_payload()
                soup = BeautifulSoup(html_content, "html.parser")
                body = soup.get_text(separator=' ')
                break
    else:
        body = msg.get_payload()
    
    return from_address, to_address, subject, date, body

In [250]:
cleaned_data = pd.DataFrame([clean_email(data["message"][i]) for i in range(len(data))], 
                            columns=['from_address', 'to_address', 'subject', 'date', 'body'])

In [251]:
cleaned_data.isnull().sum()

from_address       0
to_address      3857
subject            0
date               0
body               0
dtype: int64

In [252]:
cleaned_data = cleaned_data.dropna()
cleaned_data=cleaned_data[cleaned_data["body"]!=""]

In [253]:
cleaned_data.head()

,from_address,to_address,subject,date,body
0,phillip.allen@enron.com,tim.belden@enron.com,,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",Here is our forecast\n\n
1,phillip.allen@enron.com,john.lavorato@enron.com,Re:,"Fri, 4 May 2001 13:51:00 -0700 (PDT)",Traveling to have a business meeting takes the...
2,phillip.allen@enron.com,leah.arsdall@enron.com,Re: test,"Wed, 18 Oct 2000 03:00:00 -0700 (PDT)",test successful. way to go!!!
3,phillip.allen@enron.com,randall.gay@enron.com,,"Mon, 23 Oct 2000 06:13:00 -0700 (PDT)","Randy,\n\n Can you send me a schedule of the s..."
4,phillip.allen@enron.com,greg.piper@enron.com,Re: Hello,"Thu, 31 Aug 2000 05:07:00 -0700 (PDT)",Let's shoot for Tuesday at 11:45.


In [254]:
duplicated_rows = cleaned_data.duplicated()
print(duplicated_rows.sum())
cleaned_data = cleaned_data.drop_duplicates()
len(cleaned_data)

24546


41597

In [255]:
  
def extract_email(email_str):
    left = ''
    right = ''
    email = '@'
    
    if '@' in email_str:
        left, right = email_str.split('@', 1)
        
        
        
        for i in right:
            if i.isalnum() or (i == '.' and right[0] != '.'):
                email += i
            else:
                break
                
        for i in left[::-1]:
            if i.isalnum() or (i == '.' and left[-1] != '.'):
                email = i + email
            else:
                break      
    
    return email


In [256]:
cleaned_data['from_address'] = cleaned_data['from_address'].apply(extract_email)
cleaned_data['to_address'] = cleaned_data['to_address'].apply(extract_email)

In [257]:
print(cleaned_data[(cleaned_data['from_address'] == '@') | (cleaned_data['to_address'] == '@')])
cleaned_data = cleaned_data[(cleaned_data['from_address'] != '@') & (cleaned_data['to_address'] != '@')]
cleaned_data = cleaned_data[~(cleaned_data['from_address'].str.startswith('.') | cleaned_data['from_address'].str.endswith('.'))]
cleaned_data=cleaned_data[~(cleaned_data['to_address'].str.startswith('.') | cleaned_data['to_address'].str.endswith('.'))]
print(len(cleaned_data))
cleaned_data["date"][0].split(" ")

                     from_address to_address                   subject  \
64617  maureen.mcvicker@enron.com          @  Memo on Regulatory Risks   

                                        date  \
64617  Mon, 12 Jun 2000 01:51:00 -0700 (PDT)   

                                                    body  
64617  Please see the attached memo.\n\n\n\n\n\n\n\n\...  
41442


['Mon,', '14', 'May', '2001', '16:39:00', '-0700', '(PDT)']

In [258]:
print(len(cleaned_data[cleaned_data['date'].str[3] == ',']))
cleaned_data = cleaned_data[cleaned_data['date'].str[3] == ',']
print(len(cleaned_data))

41442
41442


In [259]:
def months():
    month=[]
    a=[]
    count=0
    for date in cleaned_data['date']:
        day, subdate = date.split(',')
        if day=='':count += count
        subdate = subdate.split(' ')
        if(len(subdate)>7) |(len(subdate)<7):count += count
        if (len(subdate)==7): 
             if not subdate[2].isalpha():
               a.append(date)
             if(subdate[2]) not in month:
              month.append(subdate[2])
    print(month)  
    print(count)  
    return a

In [260]:
a=months()
print(a)

['May', 'Oct', 'Aug', 'Jul', 'Sep', 'Apr', 'Jun', 'Mar', 'Feb', 'Jan', 'Dec', 'Nov']
0
[]


In [261]:
prefixes = ['re:', 'RE:', 'rE:']

filtered_data = cleaned_data[cleaned_data['subject'].apply(lambda x: any(x.startswith(prefix) for prefix in prefixes))]
print(len(filtered_data))
print(len(cleaned_data[cleaned_data['subject'].str.startswith('Re:')]))


5535
7077


In [262]:
def clean_subject(subject):
    prefixes = ['re:', 'RE:', 'rE:']


    for prefix in prefixes:
        if prefix in subject:
            subject = subject.replace(prefix, 'Re:', 1)
            
    prefixes = ['Fw:']

    for prefix in prefixes:
        if subject.startswith(prefix):
            subject = subject.replace(prefix, 'FW:', 1)
            break
    return subject


In [263]:
prefixes = ['fWD:',  'FwD:', 'fWd:', 'FWd:', 'FWD:', 'fwD:','fwd:','fW']
cleaned_data = cleaned_data[~cleaned_data['subject'].apply(lambda x: any(x.startswith(prefix) for prefix in prefixes))]
print(len(cleaned_data))

41437


In [264]:
cleaned_data['subject'] = cleaned_data['subject'].apply(clean_subject)
len(cleaned_data)

41437

In [265]:
prefixes = ['fWD:',  'FwD:', 'fWd:', 'FWd:', 'FWD:', 'fwD:','fwd:','fW']

filtered_data = cleaned_data[cleaned_data['subject'].apply(lambda x: any(prefix in x for prefix in prefixes)) ]
print(len(filtered_data['subject']))
cleaned_data = cleaned_data[~cleaned_data['subject'].apply(lambda x: any(prefix in x for prefix in prefixes))]

2


In [266]:
cleaned_data.duplicated().sum()
print(len(cleaned_data))


41435


In [267]:
cleaned_data = cleaned_data.drop_duplicates()
print(len(cleaned_data))

41429


In [268]:
cleaned_data.head()

,from_address,to_address,subject,date,body
0,phillip.allen@enron.com,tim.belden@enron.com,,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",Here is our forecast\n\n
1,phillip.allen@enron.com,john.lavorato@enron.com,Re:,"Fri, 4 May 2001 13:51:00 -0700 (PDT)",Traveling to have a business meeting takes the...
2,phillip.allen@enron.com,leah.arsdall@enron.com,Re: test,"Wed, 18 Oct 2000 03:00:00 -0700 (PDT)",test successful. way to go!!!
3,phillip.allen@enron.com,randall.gay@enron.com,,"Mon, 23 Oct 2000 06:13:00 -0700 (PDT)","Randy,\n\n Can you send me a schedule of the s..."
4,phillip.allen@enron.com,greg.piper@enron.com,Re: Hello,"Thu, 31 Aug 2000 05:07:00 -0700 (PDT)",Let's shoot for Tuesday at 11:45.


In [269]:
def normalize_text(text):
    text = text.replace('\n', ' ')
    #text = re.sub(r'http\S+', '', text)
    #text = re.sub(r'\S+@\S+', '', text)
    #text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    #text = text.lower()
    return text

In [270]:
cleaned_data['body'] = cleaned_data['body'].apply(normalize_text)
cleaned_data['subject'] = cleaned_data['subject'].apply(normalize_text)

In [271]:
cleaned_data.head()

,from_address,to_address,subject,date,body
0,phillip.allen@enron.com,tim.belden@enron.com,,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",Here is our forecast
1,phillip.allen@enron.com,john.lavorato@enron.com,Re:,"Fri, 4 May 2001 13:51:00 -0700 (PDT)",Traveling to have a business meeting takes the...
2,phillip.allen@enron.com,leah.arsdall@enron.com,Re: test,"Wed, 18 Oct 2000 03:00:00 -0700 (PDT)",test successful. way to go!!!
3,phillip.allen@enron.com,randall.gay@enron.com,,"Mon, 23 Oct 2000 06:13:00 -0700 (PDT)","Randy, Can you send me a schedule of the sal..."
4,phillip.allen@enron.com,greg.piper@enron.com,Re: Hello,"Thu, 31 Aug 2000 05:07:00 -0700 (PDT)",Let's shoot for Tuesday at 11:45.


In [294]:
cleaned_data['date'] = cleaned_data['date'].str.replace(r"\s\([A-Z]{3}\)", "")
cleaned_data["date"]= pd.to_datetime(cleaned_data['date'], utc=True)
print(cleaned_data.head())


C:\Users\bechi\AppData\Local\Temp\ipykernel_4476\2739242594.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  cleaned_data["date"]= pd.to_datetime(cleaned_data['date'], utc=True)


              from_address               to_address    subject  \
0  phillip.allen@enron.com     tim.belden@enron.com              
1  phillip.allen@enron.com  john.lavorato@enron.com        Re:   
2  phillip.allen@enron.com   leah.arsdall@enron.com   Re: test   
3  phillip.allen@enron.com    randall.gay@enron.com              
4  phillip.allen@enron.com     greg.piper@enron.com  Re: Hello   

                       date                                               body  
0 2001-05-14 23:39:00+00:00                            Here is our forecast     
1 2001-05-04 20:51:00+00:00  Traveling to have a business meeting takes the...  
2 2000-10-18 10:00:00+00:00                     test successful.  way to go!!!  
3 2000-10-23 13:13:00+00:00  Randy,   Can you send me a schedule of the sal...  
4 2000-08-31 12:07:00+00:00                Let's shoot for Tuesday at 11:45.    


In [295]:
cleaned_data['year'] = cleaned_data['date'].dt.year 
cleaned_data['month'] = cleaned_data['date'].dt.month
cleaned_data['day'] = cleaned_data['date'].dt.day
cleaned_data['hour'] = cleaned_data['date'].dt.hour
cleaned_data['minute'] = cleaned_data['date'].dt.minute
cleaned_data['second'] = cleaned_data['date'].dt.second
cleaned_data['day_of_week'] = cleaned_data['date'].dt.dayofweek
cleaned_data['day_name'] = cleaned_data['date'].dt.day_name()

In [298]:
cleaned_data.head()

,from_address,to_address,subject,date,body,year,month,day,hour,minute,second,day_of_week,day_name
0,phillip.allen@enron.com,tim.belden@enron.com,,2001-05-14 23:39:00+00:00,Here is our forecast,2001,5,14,23,39,0,0,Monday
1,phillip.allen@enron.com,john.lavorato@enron.com,Re:,2001-05-04 20:51:00+00:00,Traveling to have a business meeting takes the...,2001,5,4,20,51,0,4,Friday
2,phillip.allen@enron.com,leah.arsdall@enron.com,Re: test,2000-10-18 10:00:00+00:00,test successful. way to go!!!,2000,10,18,10,0,0,2,Wednesday
3,phillip.allen@enron.com,randall.gay@enron.com,,2000-10-23 13:13:00+00:00,"Randy, Can you send me a schedule of the sal...",2000,10,23,13,13,0,0,Monday
4,phillip.allen@enron.com,greg.piper@enron.com,Re: Hello,2000-08-31 12:07:00+00:00,Let's shoot for Tuesday at 11:45.,2000,8,31,12,7,0,3,Thursday
